In [1]:
import plotly.offline as pyo
import plotly_express as pyx
import plotly.graph_objects as go
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import plotly.figure_factory as ff
import numpy as np
from numpy import *
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit 

data = pd.read_csv("sample_semanai.csv")
data = data.drop_duplicates().iloc[:]
top_10 = data.groupby(['Clave platillo'])['Cantidad'].agg(
    {"Cuenta": len}).sort_values(
    "Cuenta", ascending=False).head(10).reset_index()
top_10 = top_10['Clave platillo']

/home/kaetahr/.local/lib64/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning:

using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)




In [2]:
#data_stacked = data.groupby(['Día', 'Grupo'])['Cantidad'].sum().reset_index().sort_values(['Día','Cantidad'],ascending=False)
#fig = pyx.bar(data_stacked, x='Día', y='Cantidad', color='Grupo', category_orders = {
#    'Día' : ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo' ]
#})
#fig.show()

In [3]:
#data_stacked = data.groupby(['Clave platillo', 'Grupo'])['Cantidad'].sum().reset_index().sort_values(['Clave platillo','Cantidad'],ascending=False)
#fig = pyx.bar(data_stacked, x='Grupo', y='Cantidad', color = 'Clave platillo')
#fig.show()

In [4]:
data2 = data[data["Acción"] != "Cancelación"]
data2.dropna(axis=0, subset=['Clave platillo'], inplace=True)
data_reg = data2[['Grupo', 
               'idEncuesta','El día de hoy ¿Con quién nos visita?',
               '¿Cuál es el rango de edades en tu mesa?']]
data2 = data2[['Cantidad','Clave platillo', 'idEncuesta']]
data2 = data2.dropna()

In [5]:
data_reg = data_reg.dropna()
data_reg

,Grupo,idEncuesta,El día de hoy ¿Con quién nos visita?,¿Cuál es el rango de edades en tu mesa?
0,ENTRADAS,115787,Pareja,22 a 25
1,CLÁSICOS,115787,Pareja,22 a 25
2,BEBIDAS SIN ALCOHOL,115793,Amigos,18 a 21
3,BEBIDAS SIN ALCOHOL,115793,Amigos,18 a 21
6,CLÁSICOS,115788,Pareja,26 a 32
...,...,...,...,...
46467,CLÁSICOS,615571,Pareja,18 a 21
46468,BÁSICOS,615571,Pareja,18 a 21
46469,HAMBURGUESAS,615573,Amigos,22 a 25
46470,BÁSICOS,615574,Familia,26 a 32


In [6]:
data2 = data2.groupby(['idEncuesta', 'Clave platillo'])['Cantidad'].sum().unstack().reset_index().fillna(0).set_index('idEncuesta')

In [7]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

data2 = data2.applymap(encode_units)
data2 = data2.loc[:, data2.columns.intersection(top_10)]
data2 = data2[(data2.T != 0).any()]
#data2

In [8]:
datatemp1 = data2[data2[top_10] == 1].groupby(top_10[1]).sum().reset_index().reindex(sorted(data2.columns), axis=1)
for i in range(8):
    datatemp2 = data2[data2[top_10] == 1].groupby(top_10[i+2]).sum().reset_index().reindex(sorted(data2.columns), axis=1)
    datatemp1 = datatemp1.append(datatemp2)
#datatemp1

In [9]:
datatemp1 = datatemp1.reset_index()
#datatemp1

In [10]:
datatemp1 = datatemp1[['BAS1', 'HAM12', 'MBCA63', 'BCA8', 'BAS50', 'HAM1','CLA1', 'HAM7', 'MCLÁ25']]
datatemp_tot = datatemp1.copy()
datatemp_tot['Total'] = datatemp_tot.sum(axis=1).values - 1
top_10 = top_10[1:]
datatemp_tot.set_index(top_10,inplace =True)
#datatemp1.style.background_gradient(cmap='Blues')



In [11]:
values_list = datatemp1.values.tolist()
productos = ['BAS1', 'HAM12', 'MBCA63', 'BCA8', 'BAS50', 'HAM1','CLA1', 'HAM7', 'MCLÁ25']
fig = go.Figure(data=go.Heatmap(
                    x=productos,
                    y=productos,
                    z=values_list))
#fig.show()

In [12]:
data_reg = data_reg.set_index('idEncuesta')
#data_test = pd.DataFrame(data_test)
data_reg

,Grupo,El día de hoy ¿Con quién nos visita?,¿Cuál es el rango de edades en tu mesa?
idEncuesta,,,
115787,ENTRADAS,Pareja,22 a 25
115787,CLÁSICOS,Pareja,22 a 25
115793,BEBIDAS SIN ALCOHOL,Amigos,18 a 21
115793,BEBIDAS SIN ALCOHOL,Amigos,18 a 21
115788,CLÁSICOS,Pareja,26 a 32
...,...,...,...
615571,CLÁSICOS,Pareja,18 a 21
615571,BÁSICOS,Pareja,18 a 21
615573,HAMBURGUESAS,Amigos,22 a 25


In [13]:
df_grupos = data_reg['Grupo'].str.get_dummies()
array_df = df_grupos.columns
df_grupos = df_grupos.groupby('idEncuesta')[array_df].sum().reset_index().set_index('idEncuesta')

df_por_edades = data_reg['¿Cuál es el rango de edades en tu mesa?'].str.get_dummies()
array_df = df_por_edades.columns
df_por_edades = df_por_edades.groupby('idEncuesta')[array_df].sum().reset_index().set_index('idEncuesta')

df_con_quien = data_reg['El día de hoy ¿Con quién nos visita?'].str.get_dummies()
array_df = df_con_quien.columns
df_con_quien = df_con_quien.groupby('idEncuesta')[array_df].sum().reset_index().set_index('idEncuesta')

df_chido = pd.concat([df_por_edades, df_con_quien, df_grupos], axis = 1)
df_chido = df_chido.applymap(encode_units)
df_chido

,18 a 21,22 a 25,26 a 32,33 a 44,45 ó más,Menor a 17,Amigos,Familia,Los del Trabajo,Pareja,...,DESTILADOS,ENSALADAS,ENTRADAS,EXTRAS BARRA,EXTRAS COCINA,HAMBURGUESAS,"PAPAS, PAPAS Y MÁS PAPAS",PASTAS,POSTRES,SANDWICHES
idEncuesta,,,,,,,,,,,,,,,,,,,,,
115787,0,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
115788,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
115789,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
115791,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
115793,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615470,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
615569,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
615571,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
algo = Logit(df_chido['BEBIDAS CON ALCOHOL'], sm.add_constant(df_chido[['18 a 21','22 a 25','26 a 32','33 a 44','45 ó más']]))
resultados = algo.fit()
resultados.summary()

Optimization terminated successfully.
         Current function value: 0.568574
         Iterations 5


/home/kaetahr/.local/lib64/python3.6/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                           
===============================================================================
Dep. Variable:     BEBIDAS CON ALCOHOL   No. Observations:                14836
Model:                           Logit   Df Residuals:                    14830
Method:                            MLE   Df Model:                            5
Date:                 Thu, 31 Oct 2019   Pseudo R-squ.:                0.004208
Time:                         15:32:57   Log-Likelihood:                -8435.4
converged:                        True   LL-Null:                       -8471.0
Covariance Type:             nonrobust   LLR p-value:                 5.496e-14
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.3520      0.100    -13.527      0.000      -1.548      -1.156
18 a 21        0.0486      0.110      0.443      0.658      -0.166       0.264
22 a 25        0.2700      0.105      2.566      0.010       0.064       0.476
26 a 32        0.4614      0.106      4.363      0.000       0.254       0.669
33 a 44        0.4459      0.113      3.934      0.000       0.224       0.668
45 ó más       0.3013      0.133      2.273      0.023       0.042       0.561
==============================================================================
"""

In [15]:
resultados.predict([1,1,0,0,0,0])

array([0.2135989])

In [62]:
def get_results(cant, resultados, list_x):
    results = []
    for i in range(cant):
        vector = []
        for j in range(cant):
            if j == i:
                vector.append(1)
            elif j == 0:
                vector.append(1)
            else:
                vector.append(0)
        results.append(resultados.predict(vector)[0])
    fig = pyx.scatter(x=['AMIGOS'] +list_x, y=results)
    return fig

def get_results_list(cant, resultados):
    results = []
    for i in range(cant):
        vector = []
        for j in range(cant):
            if j == i:
                vector.append(1)
            elif j == 0:
                vector.append(1)
            else:
                vector.append(0)
        results.append(resultados.predict(vector)[0])
    return results


In [17]:
resultados.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                           
===============================================================================
Dep. Variable:     BEBIDAS CON ALCOHOL   No. Observations:                14836
Model:                           Logit   Df Residuals:                    14830
Method:                            MLE   Df Model:                            5
Date:                 Thu, 31 Oct 2019   Pseudo R-squ.:                0.004208
Time:                         15:32:57   Log-Likelihood:                -8435.4
converged:                        True   LL-Null:                       -8471.0
Covariance Type:             nonrobust   LLR p-value:                 5.496e-14
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.3520      0.100    -13.527      0.000      -1.548      -1.156
18 a 21        0.0486      0.110      0.443      0.658      -0.166       0.264
22 a 25        0.2700      0.105      2.566      0.010       0.064       0.476
26 a 32        0.4614      0.106      4.363      0.000       0.254       0.669
33 a 44        0.4459      0.113      3.934      0.000       0.224       0.668
45 ó más       0.3013      0.133      2.273      0.023       0.042       0.561
==============================================================================
"""

In [25]:
#res_edad_alcohol = 
get_results(6, resultados).show()

In [ ]:
list_vals = df_con_quien.columns.to_list()[1:]
list_vals = list_vals + ['18 a 21','22 a 25','26 a 32','33 a 44','45 ó más']

alc_v_gente = Logit(df_chido['BEBIDAS CON ALCOHOL'], sm.add_constant(df_chido[list_vals]))
res_alc_v_gente = alc_v_gente.fit()
res_alc_v_gente.summary()

In [86]:
list_vals = df_con_quien.columns.to_list()[1:]

alc_v_gente = Logit(df_chido['BEBIDAS CON ALCOHOL'], sm.add_constant(df_chido[list_vals]))
res_alc_v_gente = alc_v_gente.fit()
res_alc_v_gente.summary()

Optimization terminated successfully.
         Current function value: 0.570718
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                           
===============================================================================
Dep. Variable:     BEBIDAS CON ALCOHOL   No. Observations:                14836
Model:                           Logit   Df Residuals:                    14831
Method:                            MLE   Df Model:                            4
Date:                 Thu, 31 Oct 2019   Pseudo R-squ.:               0.0004538
Time:                         16:21:48   Log-Likelihood:                -8467.2
converged:                        True   LL-Null:                       -8471.0
Covariance Type:             nonrobust   LLR p-value:                    0.1037
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.9869      0.034    -29.420      0.000      -1.053      -0.921
Familia            -0.0851      0.048     -1.758      0.079      -0.180       0.010
Los del Trabajo    -0.1044      0.105     -0.998      0.318      -0.309       0.101
Pareja             -0.1049      0.046     -2.270      0.023      -0.196      -0.014
Solo               -0.2861      0.165     -1.733      0.083      -0.610       0.037
===================================================================================
"""

In [89]:
get_results(5, res_alc_v_gente, df_con_quien.columns.to_list()[1:])

In [53]:
get_results(10, res_alc_v_gente, list_vals).show()

In [66]:
get_results_list(10, res_alc_v_gente)

[0.21971381019906355,
 0.19959817389097884,
 0.18506070611063824,
 0.2044790749218466,
 0.16976457647399468,
 0.22582672282305924,
 0.2666014113971041,
 0.3086899761484077,
 0.3080285708711702,
 0.27959502294538047]

In [83]:
list_cols = [df_con_quien.columns.to_list()] +  [df_por_edades.columns.to_list()]
def graphs_2_ints(df, lis_cols,):
    # Generar resultados
    for i in 
    regresion = Logit(df['BEBIDAS CON ALCOHOL'], sm.add_constant(df[list_cols[0]+list_cols[1]]))
    regresion = regresion.fit()
    

SyntaxError: invalid syntax (<ipython-input-83-50ea8fda5e21>, line 2)

In [82]:

list

6